In [74]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from helpers.helpers import *
import statsmodels.api as sm
import seaborn as sns
%matplotlib inline

In [75]:
emission_data = pd.read_excel("../data/Emission/verified_emissions_2018_en.xlsx", sheet_name ="verified_emissions_2018", header = 13)
treatment = pd.read_excel("../data/Emission/basic_listed_mt500.xls")

WARNING *** file size (647967) not 512 + multiple of sector size (512)


In [76]:
emission_data["INSTALLATION_NAME"] = emission_data["INSTALLATION_NAME"].apply(lambda x: standardize_string(x))
treatment["Company name"] = treatment["Company name"].apply(lambda x: standardize_string(x))

common_firms = set(emission_data["INSTALLATION_NAME"]).intersection(set(treatment["Company name"]))

treated_companies = emission_data.loc[emission_data["INSTALLATION_NAME"].isin(common_firms)]
treated_companies = treated_companies[[c for c in treated_companies.columns if "VERIFIED_EMISSIONS" in c]]
treated_companies.columns = [c.split("_")[2] for c in treated_companies.columns]

In [77]:
panel = pd.melt(treated_companies, value_vars=[c for c in treated_companies.columns], var_name ="year", value_name = "verified_emission")
panel.loc[panel.verified_emission == "Excluded"] = 0
panel = panel.loc[panel.year > 2000]
panel = panel.astype(int)

In [78]:
cutoff = 2016

panel["post_cutoff"] = (panel["year"]>cutoff).astype(int)
panel["running_variable"] = panel["year"] - cutoff
panel["intercept"] = 1

In [79]:
model = sm.OLS(panel[["verified_emission"]], panel[["intercept","post_cutoff","running_variable"]]).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      verified_emission   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.007
Method:                 Least Squares   F-statistic:                    0.1284
Date:                Tue, 07 Apr 2020   Prob (F-statistic):              0.880
Time:                        23:24:18   Log-Likelihood:                -2913.4
No. Observations:                 236   AIC:                             5833.
Df Residuals:                     233   BIC:                             5843.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
intercept         4.161e+04   7463.077      5.575      0.000    2.69e+04    5.63e+04
post_cutoff      -2009.0717   1.28e+04     -0.156      0.876   -2.73e+04    2.33e+04
running_variable   717.7465   1552.250      0.462      0.644   -2340.494    3775.987
==============================================================================
Omnibus:                      110.104   Durbin-Watson:                   2.371
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              320.965
Skew:                           2.154   Prob(JB):                     2.01e-70
Kurtosis:                       6.752   Cond. No.                         17.4
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""